# Datafolio

* Leidy Milena Alarcón Castelblanco - lalarconc@ucentral.edu.co
* Otoniel Humberto Castañeda Rodriguez - ocastanedar1@ucentral.edu.co

In [1]:
!pip install dash
!pip install jupyter-dash
!pip install plotly_express
!pip install dash-bootstrap-components

     |████████████████████████████████| 81kB 3.5MB/s 
     |████████████████████████████████| 1.0MB 5.9MB/s 
     |████████████████████████████████| 3.5MB 12.9MB/s 
     |████████████████████████████████| 194kB 39.6MB/s 
     |████████████████████████████████| 1.8MB 41.9MB/s 
     |████████████████████████████████| 358kB 48.0MB/s 
  Created wheel for dash: filename=dash-1.19.0-cp37-none-any.whl size=84011 sha256=277625092f8100b08b6670c5b6d66c099e7f3cf0c1f15628dfefc05aa764dbb7
  Stored in directory: /root/.cache/pip/wheels/f7/50/a7/a230ff7f503b10120bff18c2524a375bb85a61ce6b519c8a77
  Created wheel for dash-renderer: filename=dash_renderer-1.9.0-cp37-none-any.whl size=1014870 sha256=cd771fdf58d8a3ab898ffceeccba921db1aa1ca6bfc1a3179cbc96dc5e11f754
  Stored in directory: /root/.cache/pip/wheels/46/a0/ec/2be2e8fc750e623d76f9690c397cc5ab28b33d0a16a49e10c5
  Created wheel for dash-core-components: filename=dash_core_components-1.15.0-cp37-none-any.whl size=3527014 sha256=ed61c6b7fc133fce38d50

In [2]:
import pandas as pd
import plotly_express as px

countries_df=pd.read_csv('https://raw.githubusercontent.com/ocastanedar1/VisualizacionDatos/main/datafolio/dataset/Natural%20resources%20contribution%20to%20GDP%20by%20Country.csv')
region_df=pd.read_csv('https://raw.githubusercontent.com/ocastanedar1/VisualizacionDatos/main/datafolio/dataset/Natural%20resources%20contribution%20to%20GDP%20by%20Region.csv')
year_df=pd.read_csv('https://raw.githubusercontent.com/ocastanedar1/VisualizacionDatos/main/datafolio/dataset/Natural%20resources%20contribution%20to%20GDP%20by%20Year.csv')



In [6]:
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
import dash_table

import pandas as pd
import plotly_express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


def get_world_average_pib():
    df = year_df.copy()
    df = df.rename({'(% of GDP)': 'PIB'}, axis=1)

    df.loc[df['Indicator Name_x'] == 'Coal rents (% of GDP)', 'Indicator Name_x'] = 'Renta de carbón'
    df.loc[df['Indicator Name_x'] == 'Forest rents (% of GDP)', 'Indicator Name_x'] = 'Renta de bosques'
    df.loc[df['Indicator Name_x'] == 'Mineral rents (% of GDP)', 'Indicator Name_x'] = 'Renta de minería'
    df.loc[df['Indicator Name_x'] == 'Natural gas rents (% of GDP)', 'Indicator Name_x'] = 'Renta de gas natural'
    df.loc[df['Indicator Name_x'] == 'Oil rents (% of GDP)', 'Indicator Name_x'] = 'Renta de petróleo'
    df.loc[df['Indicator Name_x'] == 'Total natural resources rents (% of GDP)', 'Indicator Name_x'] = 'Renta total de los recursos naturales'

    fig = px.area(df[df['Country Name'] == 'World'], x="Year", y="PIB", facet_col="Indicator Name_x", facet_col_wrap=3, color='Indicator Name_x', title="Contribución de los recursos naturales al PIB a nivel mundial", labels=dict(Year="Año", PIB='% PIB'))

    fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
    fig.for_each_annotation(lambda a: a.update(text=a.text.split("(")[0]))
    fig.update_layout(showlegend=False, paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)', margin=dict(r=0))

    return fig

def get_world_contribution_pib():
    df = year_df[year_df['Indicator Name_x']=='Total natural resources rents (% of GDP)'].copy()
    df = df[~pd.isnull(df['(% of GDP)'])]
    df = df[~pd.isnull(df['Region'])]

    df = df.rename({'(% of GDP)': 'PIB', 'Country Code': 'Código', 'Country Name': 'País', 'Income Group':'Grupo de ingresos'}, axis=1)

    df.loc[df['Grupo de ingresos'] == 'Low income', 'Grupo de ingresos'] = 'Ingresos bajos'
    df.loc[df['Grupo de ingresos'] == 'Lower middle income', 'Grupo de ingresos'] = 'Ingresos medio bajos'
    df.loc[df['Grupo de ingresos'] == 'Upper middle income', 'Grupo de ingresos'] = 'Ingresos medio altos'
    df.loc[df['Grupo de ingresos'] == 'High income', 'Grupo de ingresos'] = 'Ingresos altos'

    fig = px.scatter_geo(df, locations="Código", color="Grupo de ingresos",
                        hover_name="País", size="PIB",
                        animation_frame="Year", title="Contribución anual al PIB de los recursos naturales por país", labels=dict(Year="Año", PIB='% PIB'))
    
    fig.update_geos(
        resolution=50,
        showframe=False,
        showland=True, landcolor="rgb(0, 63, 99)",
        showocean=False, bgcolor="#FFF27F",
        showcoastlines=False,
        showlakes=False,
        showrivers=False,
    )

    fig.update_layout(legend=dict(
        orientation="h",
        yanchor="bottom",
        y=0.99,
        xanchor="left",
        x=0.01,
        bgcolor='#FFF27F',
    ), height=600, paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)', margin=dict(l=0))

    return fig

def get_region_participation_pib():
    df = region_df.copy()
    df['Promedio'] = df[['2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018']].median(axis = 1, skipna = True) 
    
    df = df.rename({'Region': 'Región', 'Income Group':'Grupo de ingresos'}, axis=1)

    df.loc[df['Región'] == 'East Asia & Pacific', 'Región'] = 'Este de Asia y Pacífico'
    df.loc[df['Región'] == 'Europe & Central Asia', 'Región'] = 'Europa y Asia Central'
    df.loc[df['Región'] == 'Latin America & Caribbean', 'Región'] = 'Latinoamérica y el Caribe'
    df.loc[df['Región'] == 'Middle East & North Africa', 'Región'] = 'Medio Oriente y Norte de África'
    df.loc[df['Región'] == 'North America'] = 'Norte América'
    df.loc[df['Región'] == 'South Asia', 'Región'] = 'Sur de Asia'
    df.loc[df['Región'] == 'Sub-Saharan Africa', 'Región'] = 'África subsahariana'

    df.loc[df['Indicator Name_x'] == 'Coal rents (% of GDP)', 'Indicator Name_x'] = 'Renta de carbón'
    df.loc[df['Indicator Name_x'] == 'Forest rents (% of GDP)', 'Indicator Name_x'] = 'Renta de bosques'
    df.loc[df['Indicator Name_x'] == 'Mineral rents (% of GDP)', 'Indicator Name_x'] = 'Renta de minería'
    df.loc[df['Indicator Name_x'] == 'Natural gas rents (% of GDP)', 'Indicator Name_x'] = 'Renta de gas natural'
    df.loc[df['Indicator Name_x'] == 'Oil rents (% of GDP)', 'Indicator Name_x'] = 'Renta de petróleo'
    df.loc[df['Indicator Name_x'] == 'Total natural resources rents (% of GDP)', 'Indicator Name_x'] = 'Renta total de los recursos naturales'
        
    ind = df['Indicator Name_x'].unique().tolist()

    labels = df['Región'].unique().tolist()

    # Create subplots: use 'domain' type for Pie subplot
    #specs = [[{'type':'domain'}, {'type':'domain'}], [{'type':'domain'}, {'type':'domain'}], [{'type':'domain'}, {'type':'domain'}]]
    specs = [[{'type':'pie'}, {'type':'pie'}], [{'type':'pie'}, {'type':'pie'}], [{'type':'pie'}, {'type':'pie'}]]
    fig = make_subplots(rows=3, cols=2, specs=specs, subplot_titles=ind)

    k=1
    j=1
    for i in ind:
        dfP = df.loc[df['Indicator Name_x']==i]
        fig.add_trace(go.Pie(labels=labels, values=dfP['Promedio'], name=i),
                k, j)
        j=j+1
        if j==3:
            j=1
            k=k+1
        
        if k==4:
            k=1
            j=1
            
    fig.update_traces(hole=.4, hoverinfo="label+value", automargin=True)

    fig.update_layout(
        showlegend=True, 
        height=700,
        title_text="Contribución de los recursos naturales al PIB por región (2008-2018) ", 
        paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)',
        margin=dict(r=0, l=0, b=0),
    )

    return fig


external_stylesheets = [dbc.themes.BOOTSTRAP]

app = JupyterDash(__name__, external_stylesheets=external_stylesheets)


app.layout = dbc.Container(
    [
        html.H1("Contribución de los Recursos Naturales al Producto Interno Bruto (PIB)", style={'color': '#003F63', 'font-size':'2rem'}),
        html.P('El PIB es un indicador económico que refleja el valor monetario de todos los bienes y servicios finales producidos por un país y se utiliza para medir la riqueza que genera un país. Este datafolio presenta el porcentaje de contribución por explotación de recursos naturales (carbón, bosques, minerales, gas natural y petróleo) a la economía de cada uno de los países históricamente entre 1970 y 2018 *.', style={'color': '#212324', 'font-size':'1.2rem'}),
        html.Hr(),
        html.Br(),
        dbc.Row([
            dbc.Col([
                     html.Br(),
                     html.P("Históricamente (1970 a 2018), la contribución de los recursos naturales a nivel mundial ha estado entre el 0.89% y el 6.13%. Las rentas por bosques (único recurso renovable) se ha mantenido, las rentas por minerales y gas natural han crecido en los últimos 20 años e indudablemente el petróleo es el recurso de mayor aporte a este indicador.", style={'color': '#212324', 'font-size':'1rem'}),
                     html.P("Si bien es inquietante el crecimiento de la participación por explotación los recursos no renovables, se presume que la riqueza que se deriva de esta actividad se invierte en otras actividades productivas y sociales para no condicionar la economía futura de los países que dependen de recursos naturales finitos, de acuerdo como lo plantea la regla de Hartwick.", style={'color': '#212324', 'font-size':'1rem'}),
                ], md=3),
            dbc.Col([
                    dcc.Graph(id="world-average-pib", figure=get_world_average_pib()),         
                ], md=9),
        ]),
        html.Br(),
        html.Hr(),
        html.Br(),
        dbc.Row([
            dbc.Col(
                dcc.Graph(id="world-contribution-pib", figure=get_world_contribution_pib()),
            md=9),
            dbc.Col([
                     html.Br(),
                     html.P("En relación a la contribución total de los recursos naturales al PIB, esta varía según el país y el año. El color de la circunferencia denota el nivel de ingresos del país (alto, medio alto, medio bajo y bajo) y el tamaño de esta indica el aporte total al PIB por explotación de recursos naturales, a mayor volumen, mayor participación.", style={'color': '#212324', 'font-size':'1rem'}),
                     html.P("Como se puede observar, los ingresos de África dependen de la explotación de recursos naturales, sin embargo, su nivel de ingresos bajos y medio bajos. En contraste de lo anterior, los ingresos de los países del Golfo Pérsico se clasifican como altos a pesar de depender también de los recursos naturales.", style={'color': '#212324', 'font-size':'1rem'}),
                     html.P("De otra parte, los países con ingresos más altos en su mayoría no tienen mayor aporte a su economía por explotación de recursos naturales, como Norteamérica, Europa Occidental y China.", style={'color': '#212324', 'font-size':'1rem'}),
                     #html.Img(src="https://raw.githubusercontent.com/ocastanedar1/VisualizacionDatos/main/datafolio/img/grupo_ingresos.PNG", style={'width': '100%', 'max-width':'170px'})
                ], md=3),
        ]),
        html.Br(),
        html.Hr(),
        html.Br(),
        dbc.Row([
                 dbc.Col([
                          html.Br(),
                          html.P("A nivel de regiones en promedio durante la última década, la explotación de carbón y bosques tiene mayor aporte a la economía de Europa y Asia Central; la explotación de gas natural tiene mayor participación en la riqueza del Sur de Asia; la renta por minería está concentrada (69,5%) en Asia (Sur de Asia y Asia Central). Finalmente, la renta por explotación de petróleo se concentra en la Región de África Subsahariana.", style={'color': '#212324', 'font-size':'1rem'}),
                          html.P("En promedio total, por explotación de recursos naturales, los países de Europa y Asia Central aportan el 11,64% al PIB de su región, los países del sur de Asia aportan 8,62% a su PIB, los países de África Subsahariana aportan el 7,45% a su riqueza, mientras que los países de Latinoamérica y el Caribe aportan el 2,34% al PIB de misma región.", style={'color': '#212324', 'font-size':'1rem'})
                    ], md=3),
                dbc.Col([
                        dcc.Graph(id="region-participation-pib", figure=get_region_participation_pib())
                    ], md=9),
        ]),
        html.Br(),
        html.Hr(),
        html.Br(),
        dbc.Row([
                 dbc.Col([
                          dash_table.DataTable(
                        id='world-pib-table',
                        columns=[{'id': c, 'name': c} for c in ['Recurso Natural', '2008', '2009', '2010', '2011',
       '2012', '2013', '2014', '2015', '2016', '2017', '2018']],
                        page_size=15,  #Cantidad de elementos por página
                        style_table={
                            'backgroundColor': 'transparent',
                            'width': '100%', 
                            'overflowY': 'auto'},
                        style_header={
                            'backgroundColor': 'rgb(0, 63, 99)',
                            'color': 'rgb(255, 255, 255)',
                            'fontWeight': 'bold'
                        },
                        style_cell={
                            'backgroundColor': 'transparent',
                            'color': 'rgb(0, 63, 99)'
                        },
                        style_data={ 
                            'border': '1px solid rgb(0, 63, 99)' 
                        },
                        style_data_conditional=[
                            {
                                'if': {
                                    'filter_query': '{Recurso Natural} eq "Renta total de los recursos naturales"',
                                },
                                'backgroundColor': 'rgb(0, 63, 99)',
                                'color': 'rgb(255, 255, 255)',
                            },
                        ],
                    )
                 ], md=9),
                 dbc.Col([
                          dbc.FormGroup([
                            dbc.Label("País: "),
                            dcc.Dropdown(
                                id="country",
                                options=[
                                    {"label": col, "value": col} for col in sorted(year_df.loc[~pd.isnull(year_df['Region']), 'Country Name'].unique().tolist())
                                ],
                                value="Afghanistan",
                                searchable=False,
                                clearable=False,
                            ),
                        ]),
                        html.P("El uso de los recursos como fuente de ingresos en algunos casos depende de las ventajas geográficas de los países, Por ejemplo, los valores totales en promedio más altos en el mundo están en la región de medio Oriente y norte de África, así: Kuwait con 45,17%, Omán con 41,96%, Libia con 40,77%, Arabia Saudita con 40,37% y Qatar con 39,66%. En la siguiente tabla puede consultar la contribución de los recursos naturales al PIB por país de la última década.", style={'color': '#212324', 'font-size':'1rem'})
                    ], md=3),
        ]),
        html.Br(),
        html.Hr(),
        html.Br(),
        dbc.Row([
            dbc.Col(
                html.P('Finalmente, si bien los recursos naturales proporcionan sustento y aporte a la economía de las naciones, se hace necesario monitorear estos indicadores para evitar una excesiva dependencia económica con respecto a un determinado recurso natural agotable y en caso de que sea así, se planifique con el tiempo necesario las acciones gubernamentales para la reconversión económica (Regla de Hartwick), y de esta manera evitar la paradoja de la abundancia, que plantea “como la abundancia de recursos naturales puede hacer que el país poseedor de los mismos genere menos crecimiento económico que el país que no tenga tales recursos”.', style={'color': '#212324', 'font-size':'1.2rem'}),
            md=12),
        ]),
        html.Br(),
        html.Hr(),
        html.P("* Fuente: Banco Mundial WDI", style={'color': '#212324', 'font-size':'0.7rem'}),
        html.P("Visualización de Datos – Maestría en Analítica de Datos – Universidad Central", style={'color': '#003F63', 'text-align':'right', 'margin':'0', 'font-size':'0.7rem'}),
        html.P("Leidy Milena Alarcón - lalarconc@ucentral.edu.co", style={'color': '#003F63', 'text-align':'right', 'margin':'0', 'font-size':'0.7rem'}),
        html.P("Otoniel Humberto Castañeda - ocastanedar1@ucentral.edu.co", style={'color': '#003F63', 'text-align':'right', 'margin':'0', 'font-size':'0.7rem'}),
        html.P("2021", style={'color': '#003F63', 'text-align':'right', 'margin':'0', 'font-size':'0.7rem'}),
        
    ],
    fluid=True,
    style={'background':'#ffeb3ba6', 'padding':'4rem'},
)


@app.callback(
    Output('world-pib-table', 'data'),
    [Input('country', 'value')])
def get_world_pib_table(selected_group):
    df = year_df[year_df['Country Name'] == selected_group].pivot_table('(% of GDP)', ['Indicator Name_x'], 'Year').copy()
    df = pd.DataFrame(df.to_records())
    df = df.round(2)

    df.loc[df['Indicator Name_x'] == 'Coal rents (% of GDP)', 'Indicator Name_x'] = 'Renta de carbón'
    df.loc[df['Indicator Name_x'] == 'Forest rents (% of GDP)', 'Indicator Name_x'] = 'Renta de bosques'
    df.loc[df['Indicator Name_x'] == 'Mineral rents (% of GDP)', 'Indicator Name_x'] = 'Renta de minería'
    df.loc[df['Indicator Name_x'] == 'Natural gas rents (% of GDP)', 'Indicator Name_x'] = 'Renta de gas natural'
    df.loc[df['Indicator Name_x'] == 'Oil rents (% of GDP)', 'Indicator Name_x'] = 'Renta de petróleo'
    df.loc[df['Indicator Name_x'] == 'Total natural resources rents (% of GDP)', 'Indicator Name_x'] = 'Renta total de los recursos naturales'

    df = df.rename({'Indicator Name_x': 'Recurso Natural'}, axis=1)

    return df[['Recurso Natural', '2008', '2009', '2010', '2011',
       '2012', '2013', '2014', '2015', '2016', '2017', '2018']].to_dict('records')

app.run_server(mode='inline', debug=True)



<IPython.core.display.Javascript object>

In [8]:
#app.run_server(debug=True)